Expected input is a pyspark dataframe with a column containing image as base64 string for input into VLMs

DICOM images are previously scaled down and converted into base64 string via `dbx.pixels.dicom.dicom_to_base64jpg`

In [0]:
%pip install openai
dbutils.library.restartPython()

In [0]:
%reload_ext autoreload
%autoreload 2

In [0]:
import sys
sys.path.append('/Workspace/Users/yen.low@databricks.com/pixel/pixels')

## Load input dataframe with column containing the image as base64 string

In [0]:
df = spark.table("hls_radiology.tcia.midi_b_val_subset")
display(df)

## Run `VLMTransformer().transform(df)`

In [0]:
from dbx.pixels.dicom.dicom_vlm_phi_detector import VLMTransformer, SYSTEM_PROMPT

vlm_transformer = VLMTransformer(endpoint="databricks-claude-3-7-sonnet", 
                              system_prompt=SYSTEM_PROMPT,
                              temperature = 0.0,
                              num_output_tokens=200,
                              inputCol="jpg_base64str", 
                              outputCol="response")
out_df = vlm_transformer.transform(df.limit(3))
display(out_df)

## Run on a single image

In [0]:
from dbx.pixels.dicom.dicom_vlm_phi_detector import VLMPhiExtractor

# Get single string
image_str = df.select("jpg_base64str").take(1)[-1]['jpg_base64str']
extractor = VLMPhiExtractor(endpoint="databricks-claude-3-7-sonnet", system_prompt=SYSTEM_PROMPT)
extractor.extract(image_str)